# Collecting data from a research PACS backup drive with a manifest table

In [1]:
import pandas as pd
import os, sys
from glob import glob
import shutil
import numpy as np
import logging as log

In [2]:
## Create a logger
try:  # Python 2.7+
    from logging import NullHandler
except ImportError:
    class NullHandler(log.Handler):
        def emit(self, record):
            pass

log.getLogger().addHandler(NullHandler())

logger = log.getLogger(__name__)
ch = log.StreamHandler()
formatter = log.Formatter(fmt = '%(asctime)s %(name)s %(levelname)s: %(message)s',
                          datefmt = '%Y%m%d-%H:%M:%S')
ch.setFormatter(formatter)
logger.addHandler(ch)

# setting the log level, available levels are INFO, WARN, DEBUG
log_level = log.INFO
logger.setLevel(log_level)

In [3]:
# Location to the manifest file
fname='S:\Aimilia\Phenotype Sampling\Phenotype Sampling_with birads.xlsx'
df_final=pd.read_excel(fname, 
                       sheetname='Final Sample w location ', 
                       index_col=5) # use real accession number as the index

In [4]:
df_final.head()

,StudyId,BMI,Medview_Race,ScreenDate,Age,phenotype_sampling_all_AccessionNumber,Final Sample_PresIntentType,Final Sample_DBT,HDD,Sub_folder,Density_Overall
Final Sample_AccessionNumber,,,,,,,,,,,
8414195,1003080247,21.6,African American,2012-04-03,76.7,8414195,FOR PROCESSING,NaN,RPACS-HDD1a,data_F2,3
7673826,1001102360,36.7,African American,2011-06-30,61.4,7673826,FOR PROCESSING\,No Tomo images,RPACS-HDD1a,data_E,2
15563372,1003070850,25.3,African American,2014-08-25,54.8,15563372,FOR PROCESSING\,Raw Tomo Projection\,RPACS-HDD4a,H3_1,3
15925933,1001238977,22.4,White,2014-12-17,59.0,15925933,FOR PROCESSING,NaN,RPACS-HDD5a,E4_2,3
14345560,1000973686,26.3,African American,2013-05-23,74.1,14345560,FOR PROCESSING\,Raw Tomo Projection\,RPACS-HDD3b,F2_2\,2


In [6]:
df_final.HDD.value_counts()

RPACS-HDD4a    3840
RPACS-HDD1a    3663
RPACS-HDD3b    2863
RPACS-HDD2b    2231
RPACS-HDD5a     774
Name: HDD, dtype: int64

In [7]:
# Choose the hard drive that is currently plugged into the desktop
condition='HDD == "RPACS-HDD2b"'
df_final.query(condition).shape

(2231, 11)

In [8]:
# destination of the copying
odir=os.path.join('F:\\', 'Data', 'Penn','FOR_PROCESSING')
# find out all the studies by acc that is in HDD2b and iterate through all studies
for acc, entry in df_final.query(condition).iterrows():
    d = df_final.loc[acc]['Sub_folder']
    if not d:
        logger.warning('%s: sub_folder not specified.' % acc)
    
    # compile the paths
    copy_from = os.path.join('G:\\', d, str(acc))
    subodir=os.path.join(odir, str(acc))
    if not os.path.isdir(subodir):
        os.makedirs(subodir)
        
    # find all the files you need in the source directory (copy_from)
    # DPm* for Raw DM, DXm for Processed DM, SC* for tomo images
    files = glob(os.path.join(copy_from, 'DPm*'))
    for dcm in files:
        head, tail = os.path.split(dcm)
        # destination
        dst = os.path.join(subodir, tail)
        # check if file exists, just to be sure
        if os.path.isfile(dcm):
            # bypass files that are already transferred.
            if not os.path.isfile(dst):
                logger.info('%s -> %s' % (dcm, dst))
                shutil.copy2(dcm, dst)
            else:
                logger.warning('%s: %s already exists.' % (acc, dst))
        else:
            logger.warning('%s: %s not found' % (acc, dcm))

20180425-10:24:44 __main__ INFO: G:\H_ACC_1\8918831\DPm.1.2.840.113681.2863050707.1349247705.3500.2522 -> F:\Data\Penn\FOR_PROCESSING\8918831\DPm.1.2.840.113681.2863050707.1349247705.3500.2522
20180425-10:24:44 __main__ INFO: G:\H_ACC_1\8918831\DPm.1.2.840.113681.2863050707.1349247705.3500.2528 -> F:\Data\Penn\FOR_PROCESSING\8918831\DPm.1.2.840.113681.2863050707.1349247705.3500.2528
20180425-10:24:44 __main__ INFO: G:\H_ACC_1\8918831\DPm.1.2.840.113681.2863050707.1349247705.3500.2534 -> F:\Data\Penn\FOR_PROCESSING\8918831\DPm.1.2.840.113681.2863050707.1349247705.3500.2534
20180425-10:24:45 __main__ INFO: G:\H_ACC_1\8918831\DPm.1.2.840.113681.2863050707.1349247705.3500.2540 -> F:\Data\Penn\FOR_PROCESSING\8918831\DPm.1.2.840.113681.2863050707.1349247705.3500.2540
20180425-10:24:45 __main__ INFO: G:\H_ACC_1\8918831\DPm.1.2.840.113681.2863050707.1349247705.3500.2662 -> F:\Data\Penn\FOR_PROCESSING\8918831\DPm.1.2.840.113681.2863050707.1349247705.3500.2662
20180425-10:24:46 __main__ INFO: G:

20180425-10:25:06 __main__ INFO: G:\H_ACC_1\8914264\DPm.1.2.840.113681.2863050709.1349679889.3324.6491 -> F:\Data\Penn\FOR_PROCESSING\8914264\DPm.1.2.840.113681.2863050709.1349679889.3324.6491
20180425-10:25:06 __main__ INFO: G:\H_ACC_1\8914264\DPm.1.2.840.113681.2863050709.1349679889.3324.6516 -> F:\Data\Penn\FOR_PROCESSING\8914264\DPm.1.2.840.113681.2863050709.1349679889.3324.6516
20180425-10:25:07 __main__ INFO: G:\E2_acc_2\8516792\DPm.1.2.840.113681.2863050707.1365404613.3528.3259 -> F:\Data\Penn\FOR_PROCESSING\8516792\DPm.1.2.840.113681.2863050707.1365404613.3528.3259
20180425-10:25:07 __main__ INFO: G:\E2_acc_2\8516792\DPm.1.2.840.113681.2863050707.1365404613.3528.3366 -> F:\Data\Penn\FOR_PROCESSING\8516792\DPm.1.2.840.113681.2863050707.1365404613.3528.3366
20180425-10:25:08 __main__ INFO: G:\E2_acc_2\8516792\DPm.1.2.840.113681.2863050707.1365404613.3528.3476 -> F:\Data\Penn\FOR_PROCESSING\8516792\DPm.1.2.840.113681.2863050707.1365404613.3528.3476
20180425-10:25:09 __main__ INFO:

20180425-10:25:30 __main__ INFO: G:\E2_acc_2\14110339\DPm.1.2.840.113681.2863050711.1363158285.3132.4730 -> F:\Data\Penn\FOR_PROCESSING\14110339\DPm.1.2.840.113681.2863050711.1363158285.3132.4730
20180425-10:25:31 __main__ INFO: G:\E2_acc_2\14110339\DPm.1.2.840.113681.2863050711.1363158285.3132.4856 -> F:\Data\Penn\FOR_PROCESSING\14110339\DPm.1.2.840.113681.2863050711.1363158285.3132.4856
20180425-10:25:31 __main__ INFO: G:\E2_acc_2\14110339\DPm.1.2.840.113681.2863050711.1363158285.3132.4970 -> F:\Data\Penn\FOR_PROCESSING\14110339\DPm.1.2.840.113681.2863050711.1363158285.3132.4970
20180425-10:25:32 __main__ INFO: G:\E2_acc_1\14047983\DPm.1.2.840.113681.2863050709.1360137512.592.3101 -> F:\Data\Penn\FOR_PROCESSING\14047983\DPm.1.2.840.113681.2863050709.1360137512.592.3101
20180425-10:25:33 __main__ INFO: G:\E2_acc_1\14047983\DPm.1.2.840.113681.2863050709.1360137512.592.3215 -> F:\Data\Penn\FOR_PROCESSING\14047983\DPm.1.2.840.113681.2863050709.1360137512.592.3215
20180425-10:25:33 __main

20180425-10:25:53 __main__ INFO: G:\E2_acc_1\14078264\DPm.1.2.840.113681.2863050709.1361516764.3404.2822 -> F:\Data\Penn\FOR_PROCESSING\14078264\DPm.1.2.840.113681.2863050709.1361516764.3404.2822
20180425-10:25:54 __main__ INFO: G:\E2_acc_1\14078264\DPm.1.2.840.113681.2863050709.1361516764.3404.2842 -> F:\Data\Penn\FOR_PROCESSING\14078264\DPm.1.2.840.113681.2863050709.1361516764.3404.2842
20180425-10:25:54 __main__ INFO: G:\E2_acc_2\14113305\DPm.1.2.840.113681.2863050711.1364540359.3116.2269 -> F:\Data\Penn\FOR_PROCESSING\14113305\DPm.1.2.840.113681.2863050711.1364540359.3116.2269
20180425-10:25:55 __main__ INFO: G:\E2_acc_2\14113305\DPm.1.2.840.113681.2863050711.1364540359.3116.2367 -> F:\Data\Penn\FOR_PROCESSING\14113305\DPm.1.2.840.113681.2863050711.1364540359.3116.2367
20180425-10:25:55 __main__ INFO: G:\E2_acc_2\14113305\DPm.1.2.840.113681.2863050711.1364540359.3116.2466 -> F:\Data\Penn\FOR_PROCESSING\14113305\DPm.1.2.840.113681.2863050711.1364540359.3116.2466
20180425-10:25:56 __

20180425-10:26:17 __main__ INFO: G:\H_ACC_1\8996180\DPm.1.2.840.113681.2863050707.1351667143.3528.6548 -> F:\Data\Penn\FOR_PROCESSING\8996180\DPm.1.2.840.113681.2863050707.1351667143.3528.6548
20180425-10:26:17 __main__ INFO: G:\H_ACC_1\8996180\DPm.1.2.840.113681.2863050707.1351667143.3528.6627 -> F:\Data\Penn\FOR_PROCESSING\8996180\DPm.1.2.840.113681.2863050707.1351667143.3528.6627
20180425-10:26:18 __main__ INFO: G:\H_ACC_1\8996180\DPm.1.2.840.113681.2863050707.1351667143.3528.6709 -> F:\Data\Penn\FOR_PROCESSING\8996180\DPm.1.2.840.113681.2863050707.1351667143.3528.6709
20180425-10:26:18 __main__ INFO: G:\H_ACC_1\8996180\DPm.1.2.840.113681.2863050707.1351667143.3528.6787 -> F:\Data\Penn\FOR_PROCESSING\8996180\DPm.1.2.840.113681.2863050707.1351667143.3528.6787
20180425-10:26:19 __main__ INFO: G:\H_ACC_1\8936610\DPm.1.2.840.113681.2863050711.1349768718.3828.7427 -> F:\Data\Penn\FOR_PROCESSING\8936610\DPm.1.2.840.113681.2863050711.1349768718.3828.7427
20180425-10:26:19 __main__ INFO: G:

...

20180425-11:52:34 __main__ INFO: G:\E2_acc_2\14155150\DPm.1.2.840.113681.2863050711.1364972560.2764.3075 -> F:\Data\Penn\FOR_PROCESSING\14155150\DPm.1.2.840.113681.2863050711.1364972560.2764.3075
20180425-11:52:34 __main__ INFO: G:\E2_acc_2\14155150\DPm.1.2.840.113681.2863050711.1364972560.2764.3180 -> F:\Data\Penn\FOR_PROCESSING\14155150\DPm.1.2.840.113681.2863050711.1364972560.2764.3180
20180425-11:52:34 __main__ INFO: G:\H_ACC_1\8884546\DPm.1.2.840.113681.2863050713.1349074842.3552.1352 -> F:\Data\Penn\FOR_PROCESSING\8884546\DPm.1.2.840.113681.2863050713.1349074842.3552.1352
20180425-11:52:35 __main__ INFO: G:\H_ACC_1\8884546\DPm.1.2.840.113681.2863050713.1349074842.3552.1465 -> F:\Data\Penn\FOR_PROCESSING\8884546\DPm.1.2.840.113681.2863050713.1349074842.3552.1465
20180425-11:52:35 __main__ INFO: G:\H_ACC_1\8884546\DPm.1.2.840.113681.2863050713.1349074842.3552.1568 -> F:\Data\Penn\FOR_PROCESSING\8884546\DPm.1.2.840.113681.2863050713.1349074842.3552.1568
20180425-11:52:36 __main__ IN

20180425-11:52:56 __main__ INFO: G:\E2_acc_1\9191034\DPm.1.2.840.113681.2863050709.1359356415.3412.9062 -> F:\Data\Penn\FOR_PROCESSING\9191034\DPm.1.2.840.113681.2863050709.1359356415.3412.9062
20180425-11:52:57 __main__ INFO: G:\E2_acc_1\9191034\DPm.1.2.840.113681.2863050709.1359356415.3412.9176 -> F:\Data\Penn\FOR_PROCESSING\9191034\DPm.1.2.840.113681.2863050709.1359356415.3412.9176
20180425-11:52:57 __main__ INFO: G:\E2_acc_1\9191034\DPm.1.2.840.113681.2863050709.1359356415.3412.9296 -> F:\Data\Penn\FOR_PROCESSING\9191034\DPm.1.2.840.113681.2863050709.1359356415.3412.9296
20180425-11:52:58 __main__ INFO: G:\E2_acc_1\9191034\DPm.1.2.840.113681.2863050709.1359356415.3412.9413 -> F:\Data\Penn\FOR_PROCESSING\9191034\DPm.1.2.840.113681.2863050709.1359356415.3412.9413
20180425-11:52:58 __main__ INFO: G:\H_ACC_1\8953125\DPm.1.2.840.113681.2863050711.1351669713.3768.2660 -> F:\Data\Penn\FOR_PROCESSING\8953125\DPm.1.2.840.113681.2863050711.1351669713.3768.2660
20180425-11:52:59 __main__ INFO

20180425-11:53:16 __main__ INFO: G:\H_ACC_1\8992615\DPm.1.2.840.113681.2863050709.1352185399.3348.4436 -> F:\Data\Penn\FOR_PROCESSING\8992615\DPm.1.2.840.113681.2863050709.1352185399.3348.4436
20180425-11:53:16 __main__ INFO: G:\E2_acc_2\14218103\DPm.1.2.840.113681.2863050709.1365663690.3280.3617 -> F:\Data\Penn\FOR_PROCESSING\14218103\DPm.1.2.840.113681.2863050709.1365663690.3280.3617
20180425-11:53:17 __main__ INFO: G:\E2_acc_2\14218103\DPm.1.2.840.113681.2863050709.1365663690.3280.3725 -> F:\Data\Penn\FOR_PROCESSING\14218103\DPm.1.2.840.113681.2863050709.1365663690.3280.3725
20180425-11:53:17 __main__ INFO: G:\E2_acc_2\14218103\DPm.1.2.840.113681.2863050709.1365663690.3280.3833 -> F:\Data\Penn\FOR_PROCESSING\14218103\DPm.1.2.840.113681.2863050709.1365663690.3280.3833
20180425-11:53:18 __main__ INFO: G:\E2_acc_2\14218103\DPm.1.2.840.113681.2863050709.1365663690.3280.3851 -> F:\Data\Penn\FOR_PROCESSING\14218103\DPm.1.2.840.113681.2863050709.1365663690.3280.3851
20180425-11:53:19 __mai

20180425-11:53:37 __main__ INFO: G:\H_ACC_1\8970560\DPm.1.2.840.113681.2863050713.1350543619.3456.4043 -> F:\Data\Penn\FOR_PROCESSING\8970560\DPm.1.2.840.113681.2863050713.1350543619.3456.4043
20180425-11:53:38 __main__ INFO: G:\H_ACC_1\8970560\DPm.1.2.840.113681.2863050713.1350543619.3456.4164 -> F:\Data\Penn\FOR_PROCESSING\8970560\DPm.1.2.840.113681.2863050713.1350543619.3456.4164
20180425-11:53:38 __main__ INFO: G:\H_ACC_1\8970560\DPm.1.2.840.113681.2863050713.1350543619.3456.4310 -> F:\Data\Penn\FOR_PROCESSING\8970560\DPm.1.2.840.113681.2863050713.1350543619.3456.4310
20180425-11:53:39 __main__ INFO: G:\H_ACC_1\8970560\DPm.1.2.840.113681.2863050713.1350543619.3456.4457 -> F:\Data\Penn\FOR_PROCESSING\8970560\DPm.1.2.840.113681.2863050713.1350543619.3456.4457
20180425-11:53:39 __main__ INFO: G:\H_ACC_1\9039180\DPm.1.2.840.113681.2863050709.1352790057.3240.1536 -> F:\Data\Penn\FOR_PROCESSING\9039180\DPm.1.2.840.113681.2863050709.1352790057.3240.1536
20180425-11:53:40 __main__ INFO: G:

20180425-11:53:58 __main__ INFO: G:\H_ACC_2\9022646\DPm.1.2.840.113681.2863050711.1354001855.3332.5363 -> F:\Data\Penn\FOR_PROCESSING\9022646\DPm.1.2.840.113681.2863050711.1354001855.3332.5363
20180425-11:53:59 __main__ INFO: G:\H_ACC_2\9022646\DPm.1.2.840.113681.2863050711.1354001855.3332.5476 -> F:\Data\Penn\FOR_PROCESSING\9022646\DPm.1.2.840.113681.2863050711.1354001855.3332.5476
20180425-11:53:59 __main__ INFO: G:\E2_acc_2\14163401\DPm.1.2.840.113681.2863050707.1363762881.3464.7520 -> F:\Data\Penn\FOR_PROCESSING\14163401\DPm.1.2.840.113681.2863050707.1363762881.3464.7520
20180425-11:54:00 __main__ INFO: G:\E2_acc_2\14163401\DPm.1.2.840.113681.2863050707.1363762881.3464.7638 -> F:\Data\Penn\FOR_PROCESSING\14163401\DPm.1.2.840.113681.2863050707.1363762881.3464.7638
20180425-11:54:00 __main__ INFO: G:\E2_acc_2\14163401\DPm.1.2.840.113681.2863050707.1363762881.3464.7766 -> F:\Data\Penn\FOR_PROCESSING\14163401\DPm.1.2.840.113681.2863050707.1363762881.3464.7766
20180425-11:54:01 __main__